In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
inspector = inspect(engine)
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.

In [ ]:
MaxDate = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
MaxDate

In [ ]:
MaxDate = MaxDate[0]

In [ ]:
MaxDay = int(MaxDate[-2:])
MaxYear= int(MaxDate[:4])
MaxMonth = int(MaxDate[5:7])

In [ ]:
previous_year = dt.date(MaxYear,MaxMonth,MaxDay)-dt.timedelta(days=365)
previous_year

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
prec_query = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= previous_year).\
    order_by(Measurement.date).all()
prec_query

In [ ]:
prec_df = pd.DataFrame(prec_query)
prec_df.columns = ['date', 'precipitation']
prec_df = prec_df.set_index("date")
prec_df = prec_df.sort_index()
prec_df

In [ ]:
prec_df.plot()
plt.xticks(rotation=90)
plt.ylabel('inches')
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
#sel = [Measurement.id, Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs, Station.id,\
 #      Station.station, Station.name, Station.latitude, Station.longitude, Station.elevation]
#station_measurements = session.query(*sel).filter(Station.station == Measurement.station).all()

#station_measurements

In [ ]:
# Design a query to calculate the total number stations in the dataset
no_stations = session.query(Measurement.station).distinct().count()
print(f'There are {no_stations} stations in total.')

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

session.query(Station.id, Station.station,func.count(Measurement.id)).\
    filter(Station.station == Measurement.station).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.id).desc()).all()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

session.query(Station.id, func.max(Measurement.tobs), func.min(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Station.station == Measurement.station,Station.id == '7').\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.id).desc()).all()


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
tobs_query = session.query(Measurement.tobs).\
    filter(Station.station == Measurement.station,Station.id == '7',Measurement.date >= '2016-08-23').\
    order_by(Measurement.date).all()



In [ ]:
tobs_df = pd.DataFrame(tobs_query)
tobs_df.columns = ['tobs']
tobs_df

In [ ]:
tobs_df.hist(column='tobs', bins = 12)
#plt.xticks(np.arange(min(employeedb['salary']), max(employeedb['salary']), 10000))
plt.xticks(rotation = 90)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()

# Close Session

In [ ]:
session.close()